# Building LLMs From Scratch (Part 7): Self-Attention with Trainable Weights

Welcome to Part 7! In this notebook, we'll upgrade our simplified attention mechanism with **trainable weights**. We'll introduce the famous **Query, Key, and Value** matrices that allow the model to actually *learn* which relationships matter.

### 🔗 Quick Links
- **Medium Article**: [Part 7: Self-Attention with Trainable Weights](https://soloshun.medium.com/building-llms-from-scratch-part-7-self-attention-with-weights)
- **GitHub Repository**: [llm-from-scratch](https://github.com/soloeinsteinmit/llm-from-scratch)

### 📋 What We'll Cover
1. **The Missing Piece**: Why simplified attention can't learn
2. **Query, Key, Value**: The three trainable projections
3. **Scaled Dot-Product Attention**: Why we divide by sqrt(d_k)
4. **Step-by-Step Implementation**: Building trainable attention
5. **Two Implementations**: nn.Parameter vs nn.Linear


## Setup and Imports

Let's import the necessary libraries.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

print("✅ All imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"Device available: {'CUDA' if torch.cuda.is_available() else 'CPU'}")


## Recap: The Goal of Attention

> **The goal of self-attention is to convert `input vectors` into enriched `context vectors`.**

In Part 6, we built a simplified attention mechanism that could compute context vectors based on similarity. But it had a critical limitation: **it couldn't learn**.

The simplified version always computed attention the same way, based solely on the input embeddings. Today, we fix that by introducing **trainable weight matrices**.


In [ ]:
# Our example sentence
sentence = "Your journey starts with one step"
tokens = sentence.split()

# Input embeddings (same as Part 6)
inputs = torch.tensor([
    [0.43, 0.15, 0.89], # Your
    [0.55, 0.87, 0.66], # journey
    [0.57, 0.85, 0.64], # starts
    [0.22, 0.58, 0.33], # with
    [0.77, 0.25, 0.10], # one
    [0.05, 0.80, 0.55], # step
])

print(f"📝 Input sentence: '{sentence}'")
print(f"🔢 Tokens: {tokens}")
print(f"🧠 Input shape: {inputs.shape}")
print(f"\nInput embeddings:\n{inputs}")


## The Solution: Query, Key, and Value

The breakthrough idea: Instead of using input embeddings directly, **project them into three different spaces** using trainable weight matrices:

1. **Query (Q)**: "What am I looking for?"
2. **Key (K)**: "What do I contain?"
3. **Value (V)**: "What information do I pass on?"

### The Search Engine Analogy

Think of Google Search:
- **Query**: The text you type ("best Italian restaurants")
- **Key**: The index/metadata of web pages
- **Value**: The actual content of the pages

Google compares your **Query** to the **Keys** of all pages. If there's a strong match, it returns the corresponding **Value**.

Self-attention works the same way!


## Step 1: Initialize Weight Matrices

Let's create three trainable weight matrices: $W_q$, $W_k$, and $W_v$.


In [ ]:
# Define dimensions
d_in = inputs.shape[1]  # Input dimension = 3
d_out = 2               # Output dimension (we can project to a different size!)

print(f"📐 Dimensions:")
print(f"  Input dimension (d_in): {d_in}")
print(f"  Output dimension (d_out): {d_out}")

# Initialize weight matrices
torch.manual_seed(123)

# Note: requires_grad=False is just for this demo to keep values static
# In real training, we'd set requires_grad=True (or just use the default)
W_query = nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

print(f"\n🔧 Weight matrices initialized!")
print(f"\nW_query shape: {W_query.shape}")
print(f"W_query:\n{W_query}")

print(f"\nW_key shape: {W_key.shape}")
print(f"W_key:\n{W_key}")

print(f"\nW_value shape: {W_value.shape}")
print(f"W_value:\n{W_value}")


## Step 2: Compute Query, Key, and Value

Now we project our input embeddings into the Q, K, V spaces.


In [ ]:
# Project all inputs into Q, K, V
queries = inputs @ W_query  # (6, 3) @ (3, 2) = (6, 2)
keys    = inputs @ W_key    # (6, 3) @ (3, 2) = (6, 2)
values  = inputs @ W_value  # (6, 3) @ (3, 2) = (6, 2)

print(f"📊 Projections computed!")
print(f"\nQueries shape: {queries.shape}")
print(f"Queries:\n{queries}")

print(f"\nKeys shape: {keys.shape}")
print(f"Keys:\n{keys}")

print(f"\nValues shape: {values.shape}")
print(f"Values:\n{values}")

print(f"\n✨ We've transformed our 3D embeddings into 2D Q, K, V representations!")


## Step 3: Compute Attention Scores and Scale

Let's compute attention for "journey" and apply scaling.


In [ ]:
# Focus on "journey" (index 1)
query_idx = 1
query_2 = queries[query_idx]

print(f"🎯 Computing attention for: '{tokens[query_idx]}'")
print(f"Query vector: {query_2}")

# Compute attention scores
attn_scores_2 = query_2 @ keys.T  # (2,) @ (2, 6) = (6,)

print(f"\n📊 Attention scores (before scaling):")
for i, (token, score) in enumerate(zip(tokens, attn_scores_2)):
    print(f"  {token:>8}: {score:.4f}")

# Scale the scores
d_k = keys.shape[-1]
attn_scores_2_scaled = attn_scores_2 / (d_k ** 0.5)

print(f"\n📐 Scaling by sqrt({d_k}) = {d_k**0.5:.4f}")
print(f"\n📊 Attention scores (after scaling):")
for i, (token, score) in enumerate(zip(tokens, attn_scores_2_scaled)):
    print(f"  {token:>8}: {score:.4f}")

# Apply softmax
attn_weights_2 = torch.softmax(attn_scores_2_scaled, dim=-1)

print(f"\n🎯 Attention weights:")
for i, (token, weight) in enumerate(zip(tokens, attn_weights_2)):
    print(f"  {token:>8}: {weight:.4f} ({weight*100:.1f}%)")

# Compute context vector
context_vec_2 = attn_weights_2 @ values

print(f"\n✨ Context vector for '{tokens[query_idx]}': {context_vec_2}")


## Why Scale by sqrt(d_k)?

Let's demonstrate why scaling is crucial for training stability.


In [ ]:
# Reason 1: Preventing Softmax Saturation
tensor = torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])

print("🔬 Softmax Saturation Demo")
print("=" * 50)
print(f"Original scores: {tensor}")

# Without scaling
softmax_normal = torch.softmax(tensor, dim=-1)
print(f"\nSoftmax (normal): {softmax_normal}")
print(f"Distribution: Relatively balanced")

# With large scaling (simulating large d_k without correction)
scaled_tensor = tensor * 8
softmax_scaled = torch.softmax(scaled_tensor, dim=-1)
print(f"\nScaled scores (×8): {scaled_tensor}")
print(f"Softmax (scaled ×8): {softmax_scaled}")
print(f"Distribution: Extremely peaky! (one value dominates)")

print(f"\n💡 When scores are too large, softmax becomes overconfident.")
print(f"   This causes vanishing gradients and unstable training.")
print(f"   Dividing by sqrt(d_k) prevents this problem!")


## Computing for All Tokens at Once

Let's use matrix operations to compute attention for all tokens simultaneously.


In [ ]:
print("🚀 Computing attention for all tokens simultaneously:")
print("=" * 60)

# Compute all attention scores
attn_scores = queries @ keys.T  # (6, 2) @ (2, 6) = (6, 6)
print(f"📊 Attention scores shape: {attn_scores.shape}")

# Scale
attn_scores_scaled = attn_scores / (d_k ** 0.5)

# Apply softmax
attn_weights = torch.softmax(attn_scores_scaled, dim=-1)
print(f"🎯 Attention weights shape: {attn_weights.shape}")
print(f"Attention weights:\n{attn_weights}")

# Verify our manual calculation
print(f"\n✅ Verification - 'journey' attention weights:")
print(f"Manual: {attn_weights_2}")
print(f"Matrix: {attn_weights[1]}")
print(f"Match: {torch.allclose(attn_weights_2, attn_weights[1])}")

# Compute all context vectors
all_context_vectors = attn_weights @ values  # (6, 6) @ (6, 2) = (6, 2)
print(f"\n✨ All context vectors shape: {all_context_vectors.shape}")
print(f"All context vectors:\n{all_context_vectors}")


## Implementation 1: Using nn.Parameter

Let's wrap our attention mechanism in a clean class using `nn.Parameter`.


In [ ]:
class SelfAttention_v1(nn.Module):
    """Self-attention with trainable weights using nn.Parameter."""
    
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))
        
    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        
        attn_scores = queries @ keys.T
        attn_scores = attn_scores / keys.shape[-1]**0.5
        attn_weights = torch.softmax(attn_scores, dim=-1)
        
        context_vec = attn_weights @ values
        return context_vec

# Test it
print("🧪 Testing SelfAttention_v1:")
print("=" * 50)

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in=d_in, d_out=d_out)
output_v1 = sa_v1(inputs)

print(f"Input shape: {inputs.shape}")
print(f"Output shape: {output_v1.shape}")
print(f"\nOutput:\n{output_v1}")

# Verify it matches our manual calculation
print(f"\n✅ Matches manual calculation: {torch.allclose(output_v1, all_context_vectors)}")


## Implementation 2: Using nn.Linear (Preferred)

In practice, we use `nn.Linear` instead of `nn.Parameter`. It's cleaner, more efficient, and handles initialization automatically.


In [ ]:
class SelfAttention_v2(nn.Module):
    """Self-attention with trainable weights using nn.Linear.
    
    This is the preferred implementation - used in GPT-2, BERT, etc.
    """
    
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        
    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        
        context_vec = attn_weights @ values
        return context_vec

# Test it
print("🧪 Testing SelfAttention_v2:")
print("=" * 50)

torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in=d_in, d_out=d_out, qkv_bias=False)
output_v2 = sa_v2(inputs)

print(f"Input shape: {inputs.shape}")
print(f"Output shape: {output_v2.shape}")
print(f"\nOutput:\n{output_v2}")

print(f"\n✨ This is the foundation of GPT-2 and other transformers!")


## 🎯 Summary and Key Takeaways

We've successfully implemented self-attention with trainable weights!

### What We Accomplished:
- ✅ **Introduced Trainable Weights**: Q, K, V matrices that the model can learn
- ✅ **Understood the Analogy**: Query (search), Key (index), Value (content)
- ✅ **Implemented Scaling**: Divided by sqrt(d_k) for stability
- ✅ **Built Two Versions**: nn.Parameter (educational) and nn.Linear (practical)

### The Five-Step Process:
1. **Project into Q, K, V**: Multiply inputs by trainable weight matrices
2. **Compute Attention Scores**: Dot product between queries and keys
3. **Scale**: Divide by sqrt(d_k) for stability
4. **Normalize**: Apply softmax to get attention weights
5. **Weighted Sum**: Multiply attention weights by values

### Key Insights:
- **Trainable weights** allow the model to learn optimal attention patterns
- **Query** determines what to look for
- **Key** represents what content is available
- **Value** contains the actual information to retrieve
- **Scaling** prevents softmax saturation and stabilizes variance
- **nn.Linear** is preferred over nn.Parameter in practice

### What's Next:
In **Part 8**, we'll implement **Causal Attention** (Masked Self-Attention) to ensure the model can only look at previous tokens, not future ones. This is crucial for text generation!

---

### 🔗 Resources
- **Medium Article**: [Part 7: Self-Attention with Trainable Weights](https://soloshun.medium.com/building-llms-from-scratch-part-7-self-attention-with-weights)
- **GitHub Repository**: [llm-from-scratch](https://github.com/soloeinsteinmit/llm-from-scratch)
- **Python Script**: `src/part07_self_attention_with_weights.py`

### 🔜 Coming Next
**Part 8: Causal Attention (Masked Self-Attention)**
